In [ ]:
import numpy as np
import bilby
import matplotlib.pyplot as plt
import pandas as pd
duration = 4
sampling_frequency = 2048.
outdir = 'outdir'
label = 'fishermatrixerr'
bilby.core.utils.setup_logger(outdir=outdir, label=label)


#waveform_generator
# Create the waveform_generator using a LAL BinaryBlackHole source function
waveform_arguments = dict(waveform_approximant='IMRPhenomPv2',
                          reference_frequency=50., minimum_frequency=20.)
waveform_generator = bilby.gw.WaveformGenerator(
        duration=duration, sampling_frequency=sampling_frequency,
        frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
        waveform_arguments=waveform_arguments)

#calculate the polorization signal with 15 parameter and waveform_generator
#return h+ hx data array
def insig(mass_1,mass_2, a_1, a_2, tilt_1, tilt_2,phi_12, phi_jl, luminosity_distance, theta_jn, psi,phase, geocent_time, ra, dec,waveform_generator=waveform_generator):
    injection_parameters=dict(mass_1=mass_1,mass_2=mass_2, a_1=a_1, a_2=a_2, tilt_1=tilt_1, tilt_2=tilt_2,phi_12=phi_12,\
                              phi_jl=phi_jl, luminosity_distance=luminosity_distance, theta_jn=theta_jn, psi=psi,phase=phase, geocent_time=geocent_time, ra=ra, dec=dec)
    ifos = bilby.gw.detector.InterferometerList(['ET'])
    ifos.set_strain_data_from_power_spectral_densities( ##power spectral
        sampling_frequency=sampling_frequency, duration=duration,
        start_time=injection_parameters['geocent_time'] - 3)
    return ifos.inject_signal(waveform_generator=waveform_generator,parameters=injection_parameters)

#calculate the derivative for dl m1 m2 l
#return dh/dDL, dh/dm1,dh/dm2,dh/dl 
def derivative(table,param,tol):
    h_pola=insig(table['mass_1'],table['mass_2'],table['a_1'],table['a_2'],table['tilt_1'],table['tilt_2'],table['phi_12'],\
          table['phi_jl'],table['luminosity_distance'],table['theta_jn'],table['psi'],table['phase'],\
          table['geocent_time'],table['ra'],table['dec'],waveform_generator=waveform_generator)
    if param =='mass_1':
         h_pola_new=insig(table['mass_1']+tol,table['mass_2'],table['a_1'],table['a_2'],table['tilt_1'],table['tilt_2'],table['phi_12'],\
          table['phi_jl'],table['luminosity_distance'],table['theta_jn'],table['psi'],table['phase'],\
          table['geocent_time'],table['ra'],table['dec'],waveform_generator=waveform_generator) 
    elif param =='mass_2':
         h_pola_new=insig(table['mass_1'],table['mass_2']+tol,table['a_1'],table['a_2'],table['tilt_1'],table['tilt_2'],table['phi_12'],\
          table['phi_jl'],table['luminosity_distance'],table['theta_jn'],table['psi'],table['phase'],\
          table['geocent_time'],table['ra'],table['dec'],waveform_generator=waveform_generator) 
    elif param =='luminosity_distance':
         h_pola_new=insig(table['mass_1'],table['mass_2'],table['a_1'],table['a_2'],table['tilt_1'],table['tilt_2'],table['phi_12'],\
          table['phi_jl'],table['luminosity_distance']+tol,table['theta_jn'],table['psi'],table['phase'],\
          table['geocent_time'],table['ra'],table['dec'],waveform_generator=waveform_generator) 
    elif param=='theta_jn':
        h_pola_new=insig(table['mass_1'],table['mass_2'],table['a_1'],table['a_2'],table['tilt_1'],table['tilt_2'],table['phi_12'],\
          table['phi_jl'],table['luminosity_distance'],table['theta_jn']+tol,table['psi'],table['phase'],\
          table['geocent_time'],table['ra'],table['dec'],waveform_generator=waveform_generator)       
    else:
        raise ValueError('Parameter not supported!')
    return (h_pola_new[0]['plus']- h_pola[0]['plus']) / tol

#calculate the inner_product for two derivative <dh/dxi|dh/dxj> adding the ET psd
def inner_product(aa, bb, power_spectral_density, duration):
    integrand = np.conj(aa) * bb / power_spectral_density
    return 4 / duration * np.sum(integrand)

#using inner_product to calculate the fisher information matrix(fim)
#return 4x4 matrix
def calculate_fisher(parameters,table,tolerance,power_spectral_density,duration):
    '''
    parameters = array of the name of the waveform parameters to be included in the analysis
    table = dictionary of waveform parameters
    tolerance = step of the waveform derivative   
    returns the errors on the input parameters 4x4 parameters
    '''
    fim = np.zeros([len(parameters),len(parameters)])
    for i,val1 in enumerate(parameters):
        for j,val2 in enumerate(parameters):
            fim[i][j] = inner_product(derivative(table,val1,tolerance),derivative(table,val2,tolerance),\
                             ifos[0].power_spectral_density_array, duration).real
    return fim 

#calculate the mock data's fim
#15 parameter + snr for mock data
inpa=np.loadtxt('LCDMcosmology15p.txt')
mockdata=np.zeros((len(inpa),22))
for i in range(len(inpa)):
    injection_parameters_k= dict(
        mass_1=inpa[i,0], mass_2=inpa[i,1], a_1=inpa[i,2], a_2=inpa[i,3], tilt_1=inpa[i,4], tilt_2=inpa[i,5],
        phi_12=inpa[i,6], phi_jl=inpa[i,7], luminosity_distance=inpa[i,8], theta_jn=inpa[i,9], psi=inpa[i,10],
        phase=inpa[i,11], geocent_time=inpa[i,12], ra=inpa[i,13], dec=inpa[i,14])
    ifos = bilby.gw.detector.InterferometerList(['ET'])
    ifos.set_strain_data_from_power_spectral_densities( ##power spectral
        sampling_frequency=sampling_frequency, duration=duration,
        start_time=injection_parameters_k['geocent_time'] - 3)
    ifos.inject_signal(waveform_generator=waveform_generator,
                   parameters=injection_parameters_k)
    table = {}
    table['mass_1']             =inpa[i,0]
    table['mass_2']             =inpa[i,1]
    table['a_1']                =inpa[i,2]
    table['a_2']                =inpa[i,3]
    table['tilt_1']             =inpa[i,4]
    table['tilt_2']             =inpa[i,5]
    table['phi_12']             =inpa[i,6]
    table['phi_jl']             =inpa[i,7]
    table['luminosity_distance']=inpa[i,8]
    table['theta_jn']           =inpa[i,9]
    table['psi']                =inpa[i,10]
    table['phase']              =inpa[i,11]
    table['geocent_time']       =inpa[i,12]
    table['ra']                 =inpa[i,13]
    table['dec']                =inpa[i,14]

    h_pola=insig(table['mass_1'],table['mass_2'],table['a_1'],table['a_2'],table['tilt_1'],table['tilt_2'],table['phi_12'],\
          table['phi_jl'],table['luminosity_distance'],table['theta_jn'],table['psi'],table['phase'],\
          table['geocent_time'],table['ra'],table['dec'],waveform_generator=waveform_generator)
    #step for derivative
    tol = 10**(-11)
    #calculate the m1 m2 DL l derivative
    params = ['mass_1','mass_2','luminosity_distance','theta_jn']
    fim = calculate_fisher(params,table,tol,ifos[0].power_spectral_density_array,duration)
    #[fimatrix]^-1 
    Sigma = np.linalg.inv(fim) 
    #Sqrt the Diagonal 
    sqrtt=np.sqrt(np.diag(Sigma))
    #save fisher matrix err for dl m1 m2 l
    mockdata[i,15]=sqrtt[0]
    mockdata[i,16]=sqrtt[1]
    mockdata[i,17]=sqrtt[2]
    mockdata[i,18]=sqrtt[3]
    #save 15 injection parameters
    mockdata[i,0] =inpa[i,0]
    mockdata[i,1] =inpa[i,1]
    mockdata[i,2] =inpa[i,2]
    mockdata[i,3] =inpa[i,3]
    mockdata[i,4] =inpa[i,4]
    mockdata[i,5] =inpa[i,5]
    mockdata[i,6] =inpa[i,6]
    mockdata[i,7] =inpa[i,7]
    mockdata[i,8] =inpa[i,8]
    mockdata[i,9] =inpa[i,9]
    mockdata[i,10]=inpa[i,10]
    mockdata[i,11]=inpa[i,11]
    mockdata[i,12]=inpa[i,12]
    mockdata[i,13]=inpa[i,13]
    mockdata[i,14]=inpa[i,14]
    mockdata[i,19]=ifos[0].meta_data['matched_filter_SNR']
    mockdata[i,20]=ifos[0].meta_data['optimal_SNR']
    mockdata[i,21]=sqrtt[2]/inpa[i,8]
np.savetxt('LCDMmockdata_fim_snr.txt',mockdata)